In [4]:
import pandas as pd
df_ori=pd.read_excel("data.xlsx")
df=df_ori[df_ori.account==3].copy().reset_index()
df[0:10]


,index,id,line,account,date_trans,date_entry,dc,amount,seed
0,33,8,4,3,2020-07-05,2020-07-05,C,-27409.79,0
1,67,15,3,3,2020-07-06,2020-07-08,D,20822.39,0
2,69,16,1,3,2020-07-06,2020-07-09,C,-47566.41,0
3,89,20,1,3,2020-07-08,2020-07-12,C,-13364.02,0
4,91,21,1,3,2020-07-08,2020-07-12,C,-30669.44,0
5,111,26,3,3,2020-07-14,2020-07-15,D,28301.23,0
6,115,27,3,3,2020-07-14,2020-07-15,C,-46480.74,0
7,156,35,7,3,2020-07-17,2020-07-17,D,6120.01,0
8,168,38,5,3,2020-07-16,2020-07-18,D,17234.82,0
9,173,39,4,3,2020-07-16,2020-07-18,D,32569.70,0


In [5]:
field_to_check='amount'
list_to_check=df[field_to_check].to_list()
list_to_check=[int(round(x,2)*100) for x in list_to_check]
matched=[False for n in range(len(list_to_check))]
tuples=list(zip(df.index,list_to_check))
debs_pos=[x[0] for x in tuples if x[1] > 0]
debs_amt=[x[1] for x in tuples if x[1] > 0]
creds_pos=[x[0] for x in tuples if x[1] < 0]
creds_amt=[x[1] for x in tuples if x[1] < 0]
print(list_to_check[:10])
print(debs_amt[0:10])
print(creds_amt[0:10])

[-2740979, 2082239, -4756641, -1336402, -3066944, 2830123, -4648074, 612001, 1723482, 3256970]
[2082239, 2830123, 612001, 1723482, 3256970, 2740979, 2301566, 3371741, 3959303, 3884983]
[-2740979, -4756641, -1336402, -3066944, -4648074, -1016859, -3106413, -2301566, -2028368, -2830123]


In [6]:
def check_match_list():
    running=0
    for n in match_list:
        tot=0
        for m in n:
            for v in m:
                tot=tot+df.iloc[v]['amount']
        if round(tot,1)==0:
            running=running+1
    print(running,len(match_list))

In [7]:
def find_possible_matches(v,list_amounts,ref="", list_refs=[]):
    matches=[]
    pos=[]
    ref=[]
    start=0
    while True:
        try:
            p=list_amounts.index(v,start)
            pos.append(p)
            start=p+1
        except Exception as e:
            break
    if len(pos)==1:
        return pos[0],pos, 1
    if len(pos)==0:
        return False, False, False
    if len(pos)>1:
        return pos[0], pos, len(pos)  
        #here we would develop something to choose, either least tolerance, or first/last 
        # or using regexp to select based on a potential reference field  
test=[1,3,200,100,1,3,200.01,200.001, 200.0001]
find_possible_matches(200,test) 

(2, [2], 1)

In [8]:
import traceback
match_list=[]
for i, v in enumerate(list_to_check):
    if v<0: #it is a credit, we search for debits that match
        try:      
            pos_deb, matched_pos, matched_count =find_possible_matches(-v, debs_amt) #debs_amt.index(-v)
            if pos_deb==False:
                continue    
            matched[debs_pos[pos_deb]]=True
            matched[i]=True
            match_list.append([[debs_pos[pos_deb]],[i]])
            debs_pos.pop(pos_deb)
            debs_amt.pop(pos_deb)        
            pointer=creds_pos.index(i)
            creds_pos.pop(pointer)
            creds_amt.pop(pointer)
        except Exception as e:
            if type(e).__name__ == 'ValueError':
                pass
            else:
                print(e)
                print(traceback.format_exc())
                break
    else:
        try:
            pos_cred, matched_pos, matched_count=find_possible_matches(-v, creds_amt) # creds_amt.index(-v)          
            if pos_cred==False:
                continue
            matched[i]=True
            matched[creds_pos[pos_cred]]=True
            match_list.append([[i],[creds_pos[pos_cred]]])         
            pointer=debs_pos.index(i)
            debs_pos.pop(pointer)
            debs_amt.pop(pointer)         
            creds_pos.pop(pos_cred)
            creds_amt.pop(pos_cred)        
        except Exception as e:
            if type(e).__name__ == 'ValueError':
                pass
            else:
                print(e)              
                print(traceback.format_exc())               
                break
match_list[0:4]

[[[10], [0]], [[1], [83]], [[20], [2]], [[121], [3]]]

In [9]:
check_match_list()

198 198


In [10]:
print(len(debs_pos))
print(len(creds_pos))
print(len(list_to_check),len(debs_pos)+len(creds_pos))

24
30
450 54


In [11]:
from itertools import combinations
deb_perm_list=list(combinations(debs_pos,2))
cred_perm_list=list(combinations(creds_pos,2))


In [12]:
deb_perm_list_sum=[]
for x in deb_perm_list:
    try:   
        deb_perm_list_sum.append(list_to_check[x[0]]+list_to_check[x[1]])
    except:
        print("Error with ", x)
        break
cred_perm_list_sum=[]
for x in cred_perm_list:
    try:   
        cred_perm_list_sum.append(list_to_check[x[0]]+list_to_check[x[1]])
    except:
        print("Error with ", x)
        break


In [13]:
for i, v in enumerate(list_to_check):
    if matched[i]==False:
        if v<0:
            try:
                #here we search the credit in the list of combinations of debits taken by 2, summed
                pos_deb, matched_pos, matched_count =find_possible_matches(-v,deb_perm_list_sum)
                if pos_deb==False:
                    continue
                    
                matched[i]=True
                matched[deb_perm_list[pos_deb][0]]=True
                matched[deb_perm_list[pos_deb][1]]=True             
                match_list.append([list(deb_perm_list[pos_deb]),[i]])
                
                #we remove the matched elements from the list of sums and their pointers 
                          
                debs_pointer1=debs_pos.index(deb_perm_list[pos_deb][0])
                debs_pos.pop(debs_pointer1)
                debs_amt.pop(debs_pointer1)
                
                debs_pointer2=debs_pos.index(deb_perm_list[pos_deb][1])
                debs_pos.pop(debs_pointer2)
                debs_amt.pop(debs_pointer2)
                
                creds_pointer=creds_pos.index(i)
                creds_pos.pop(creds_pointer)
                creds_amt.pop(creds_pointer)
                
                deb_perm_list_sum.pop(pos_deb)
                deb_perm_list.pop(pos_deb)
                
            except Exception as e:
                if type(e).__name__ == 'ValueError':
                    pass
                else:
                    print(e)
                    break
                
        else:
            try:
                pos_cred, matched_pos, matched_count =find_possible_matches(-v,cred_perm_list_sum)
                if pos_cred==False:
                    continue          
                matched[i]=True         
                matched[cred_perm_list[pos_cred][0]]=True
                matched[cred_perm_list[pos_cred][1]]=True           
                match_list.append([[i], list(cred_perm_list[pos_cred])])
                
                creds_pointer1=creds_pos.index(cred_perm_list[pos_cred][0])
                creds_pos.pop(creds_pointer1)
                creds_amt.pop(creds_pointer1)
                
                creds_pointer2=creds_pos.index(cred_perm_list[pos_cred][1])
                creds_pos.pop(creds_pointer2)
                creds_amt.pop(creds_pointer2)
                
                debs_pointer=debs_pos.index(i)
                debs_pos.pop(debs_pointer)
                debs_amt.pop(debs_pointer)
                
                cred_perm_list_sum.pop(pos_cred)
                cred_perm_list.pop(pos_cred)
                
                
                
            except Exception as e:
                if type(e).__name__ == 'ValueError':
                    pass
                else:
                    print(e)
                    break

In [14]:
match_list[-20:]

[[[431], [399]],
 [[403], [439]],
 [[404], [429]],
 [[409], [446]],
 [[410], [447]],
 [[440], [411]],
 [[448], [415]],
 [[445], [417]],
 [[436], [418]],
 [[77, 78], [45]],
 [[107], [157, 158]],
 [[129, 130], [120]],
 [[240, 241], [156]],
 [[160], [291, 292]],
 [[210, 211], [183]],
 [[232], [306, 307]],
 [[267], [333, 334]],
 [[270], [405, 406]],
 [[277], [310, 311]],
 [[419], [441, 442]]]

In [15]:
print("% pending to reconcile: ",(len(debs_amt)+len(creds_amt))/len(list_to_check))

% pending to reconcile:  0.04666666666666667


In [16]:
check_match_list()

209 209


In [20]:
import numpy as np
matched_id=[np.nan for n in range(len(list_to_check))]
for i, t in enumerate(match_list):
    m=t[0]+t[1]
    for n in m:
        matched_id[n]=i+1
df1=pd.DataFrame(zip(matched, matched_id),columns=["matched","matched_id"])
df1.head()

,matched,matched_id
0,True,1.0
1,True,2.0
2,True,3.0
3,True,4.0
4,True,5.0


In [21]:
df2=pd.merge(df,df1, left_index=True, right_index=True)

In [25]:
df2[df2.matched==True].sort_values("matched_id", ascending=True)

,index,id,line,account,date_trans,date_entry,dc,amount,seed,matched,matched_id
0,33,8,4,3,2020-07-05,2020-07-05,C,-27409.79,0,True,1.0
10,195,43,8,3,2020-07-24,2020-07-19,D,27409.79,1,True,1.0
1,67,15,3,3,2020-07-06,2020-07-08,D,20822.39,0,True,2.0
83,1033,220,1,3,2020-07-20,2020-10-03,C,-20822.39,1,True,2.0
2,69,16,1,3,2020-07-06,2020-07-09,C,-47566.41,0,True,3.0
...,...,...,...,...,...,...,...,...,...,...,...
311,3496,732,4,3,2021-04-16,2021-04-01,C,-1263.20,2,True,208.0
310,3495,732,3,3,2021-04-16,2021-04-01,C,-1217.57,2,True,208.0
441,4712,992,1,3,2021-07-02,2021-07-02,C,-292.94,2,True,209.0
442,4713,992,2,3,2021-07-02,2021-07-02,C,-662.46,2,True,209.0


In [23]:
df2.to_excel("matched.xlsx")

In [37]:
df3=pd.merge(df_ori,df2[['index','matched','matched_id']],left_index=True, right_on="index", how='left')
df3.to_excel("matched_ori.xlsx")

,id,line,account,date_trans,date_entry,dc,amount,seed,index,matched,matched_id
0.0,8,4,3,2020-07-05,2020-07-05,C,-27409.79,0,33,True,1.0
1.0,15,3,3,2020-07-06,2020-07-08,D,20822.39,0,67,True,2.0
2.0,16,1,3,2020-07-06,2020-07-09,C,-47566.41,0,69,True,3.0
3.0,20,1,3,2020-07-08,2020-07-12,C,-13364.02,0,89,True,4.0
4.0,21,1,3,2020-07-08,2020-07-12,C,-30669.44,0,91,True,5.0
5.0,26,3,3,2020-07-14,2020-07-15,D,28301.23,0,111,True,6.0
6.0,27,3,3,2020-07-14,2020-07-15,C,-46480.74,0,115,True,7.0
7.0,35,7,3,2020-07-17,2020-07-17,D,6120.01,0,156,False,NaN
8.0,38,5,3,2020-07-16,2020-07-18,D,17234.82,0,168,True,8.0
9.0,39,4,3,2020-07-16,2020-07-18,D,32569.70,0,173,True,9.0
